# Parviflo — LIVE Data Notebook (OBIS + ERDDAP)

Pulls **real** jellyfish occurrences (OBIS) and **real** satellite/environment layers (ERDDAP).
All visuals are produced here with Matplotlib/Cartopy (no separate front-end).

In [1]:
import os, warnings
from typing import Tuple, List
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# Cartopy is optional; maps will fall back to plain imshow if not installed
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    HAS_CARTOPY = True
except Exception:
    HAS_CARTOPY = False

# Region and grid
BBOX = (-6.0, 30.0, 36.0, 46.0)  # min_lon, min_lat, max_lon, max_lat (Mediterranean)
GRID_DEG = 0.25

# Time window: last 12 full months
END = (pd.Timestamp.utcnow() - pd.offsets.MonthEnd(1)).normalize()
START = END - pd.DateOffset(months=11)
MONTHS = pd.period_range(START.to_period('M'), END.to_period('M'), freq='M')

print('BBOX:', BBOX)
print('Months:', MONTHS[0], 'to', MONTHS[-1])

BBOX: (-6.0, 30.0, 36.0, 46.0)
Months: 2024-10 to 2025-09


## 1) OBIS — Jellyfish occurrences (Scyphozoa)

We query the OBIS v3 API within the bbox and aggregate per 0.25° cell per month.

In [ ]:
OBIS_BASE = 'https://api.obis.org/v3/occurrence'

def obis_monthly_counts(bbox: Tuple[float,float,float,float], year:int, month:int, taxon_class='Scyphozoa', page_size=2000):
    """Return DataFrame [lat_bin, lon_bin, jelly_n] for (year, month) from OBIS."""
    poly = f"POLYGON(({bbox[0]} {bbox[1]},{bbox[2]} {bbox[1]},{bbox[2]} {bbox[3]},{bbox[0]} {bbox[3]},{bbox[0]} {bbox[1]}))"
    start = pd.Timestamp(year=year, month=month, day=1)
    end = (start + pd.offsets.MonthEnd(1)).date().isoformat()

    offset = 0
    recs = []
    while True:
        params = dict(
            geometry=poly,
            startdate=start.date().isoformat(),
            enddate=end,
            Class=taxon_class,
            size=page_size,
            _from=offset
        )
        r = requests.get(OBIS_BASE, params=params, timeout=60)
        r.raise_for_status()
        data = r.json().get('results', [])
        if not data:
            break
        recs.extend(data)
        if len(data) < page_size:
            break
        offset += page_size

    if not recs:
        return pd.DataFrame(columns=['lat_bin','lon_bin','jelly_n'])

    df = pd.DataFrame(recs)
    df = df.dropna(subset=['decimalLatitude','decimalLongitude'])
    df['lat_bin'] = np.floor(df['decimalLatitude']/GRID_DEG)*GRID_DEG
    df['lon_bin'] = np.floor(df['decimalLongitude']/GRID_DEG)*GRID_DEG
    out = df.groupby(['lat_bin','lon_bin']).size().reset_index(name='jelly_n')
    return out

# Pull last 12 months
jelly_monthly = []
for p in MONTHS:
    y, m = p.year, p.month
    print(f'OBIS {y}-{m:02d}...')
    try:
        c = obis_monthly_counts(BBOX, y, m)
        c['year'] = y; c['month'] = m
        jelly_monthly.append(c)
    except Exception as e:
        print('  OBIS error:', e)

jelly_df = pd.concat(jelly_monthly, ignore_index=True) if jelly_monthly else pd.DataFrame(columns=['lat_bin','lon_bin','jelly_n','year','month'])
jelly_df.head()

OBIS 2024-10...


## 2) ERDDAP — Chlorophyll & SST

- Chlorophyll (OLCI global 4km): `noaacwS3AOLCIchlaDaily`
- SST (OISST 0.25°): `ncdc_oisst_v2_avhrr_by_time_zlev_lat_lon`

We convert daily data to monthly means and subset to the bbox.

In [ ]:
import xarray as xr

ERDDAP_CHLA = 'https://coastwatch.noaa.gov/erddap/griddap/noaacwS3AOLCIchlaDaily.nc'
ERDDAP_OISST = 'https://www.ncei.noaa.gov/erddap/griddap/ncdc_oisst_v2_avhrr_by_time_zlev_lat_lon.nc'

def monthly_field(ds_url: str, start: str, end: str, bbox: Tuple[float,float,float,float]):
    ds = xr.open_dataset(ds_url)
    # coord names
    lat_name = 'latitude' if 'latitude' in ds.coords else 'lat'
    lon_name = 'longitude' if 'longitude' in ds.coords else 'lon'
    tname = 'time'
    sub = ds.sel({tname: slice(start, end)})
    lmin, lmax = bbox[0], bbox[2]
    if float(sub[lon_name].max()) > 180:
        lmin = (lmin + 360) % 360
        lmax = (lmax + 360) % 360
    sub = sub.sel({lat_name: slice(bbox[1], bbox[3]), lon_name: slice(lmin, lmax)})
    var = list(sub.data_vars)[0]
    return sub[var].resample({tname: '1MS'}).mean()

chla_m = monthly_field(ERDDAP_CHLA, str(MONTHS[0].start_time.date()), str(MONTHS[-1].end_time.date()), BBOX)
sst_m  = monthly_field(ERDDAP_OISST, str(MONTHS[0].start_time.date()), str(MONTHS[-1].end_time.date()), BBOX)

chla_m, sst_m

## 3) Features, labels, and model (logistic)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, brier_score_loss

# Grid cells
lats = np.arange(BBOX[1], BBOX[3]+1e-9, GRID_DEG)
lons = np.arange(BBOX[0], BBOX[2]+1e-9, GRID_DEG)
cells = pd.MultiIndex.from_product([np.round(lats,6), np.round(lons,6)], names=['lat_bin','lon_bin']).to_frame(index=False)

# months table
T = pd.DataFrame([(p.year, p.month) for p in MONTHS], columns=['year','month'])
mesh = cells.merge(T, how='cross')

# JellyIdx
j = jelly_df.copy()
j['JellyIdx'] = np.log1p(j['jelly_n'])
mesh = mesh.merge(j[['lat_bin','lon_bin','year','month','JellyIdx']], on=['lat_bin','lon_bin','year','month'], how='left')
mesh['JellyIdx'] = mesh['JellyIdx'].fillna(0.0)

# Convert DataArrays to tables
def da_to_df(da, name):
    df = da.to_dataframe(name=name).reset_index()
    lat_name = 'latitude' if 'latitude' in df.columns else 'lat'
    lon_name = 'longitude' if 'longitude' in df.columns else 'lon'
    df['year'] = pd.to_datetime(df['time']).dt.year
    df['month'] = pd.to_datetime(df['time']).dt.month
    df = df.rename(columns={lat_name: 'lat_bin', lon_name: 'lon_bin'})
    return df[['lat_bin','lon_bin','year','month', name]]

mesh = mesh.merge(da_to_df(chla_m, 'chla'), on=['lat_bin','lon_bin','year','month'], how='left')
mesh = mesh.merge(da_to_df(sst_m, 'sst'),  on=['lat_bin','lon_bin','year','month'], how='left')

# Simple climatology = first 3 months per cell
def anomaly_by_cell(df, col, k=3):
    base = df.groupby(['lat_bin','lon_bin'])[col].apply(lambda s: s.iloc[:k].mean()).reset_index().rename(columns={col: f'{col}_clim'})
    out = df.merge(base, on=['lat_bin','lon_bin'], how='left')
    out[f'{col}_anom'] = out[col] - out[f'{col}_clim']
    return out

mesh = anomaly_by_cell(mesh, 'sst')
mesh = anomaly_by_cell(mesh, 'chla')

mesh = mesh.sort_values(['lat_bin','lon_bin','year','month']).reset_index(drop=True)
mesh['flip_now'] = (mesh['JellyIdx'] > mesh.groupby(['lat_bin','lon_bin'])['JellyIdx'].transform(lambda s: s.quantile(0.7))).astype(int)

def make_samples(df, lags=6, fmin=3, fmax=6):
    rows = []
    for (la,lo), g in df.groupby(['lat_bin','lon_bin']):
        g = g.reset_index(drop=True)
        for i in range(lags, len(g)-fmax):
            past = g.iloc[i-lags:i]
            fut  = g.iloc[i+fmin:i+fmax+1]
            feat = {
                'lat_bin': la, 'lon_bin': lo,
                'JellyIdx_last': past['JellyIdx'].iloc[-1],
                'JellyIdx_mean': past['JellyIdx'].mean(),
                'sst_anom_last': past['sst_anom'].iloc[-1],
                'sst_anom_mean': past['sst_anom'].mean(),
                'chla_anom_last': past['chla_anom'].iloc[-1],
                'chla_anom_mean': past['chla_anom'].mean(),
            }
            y = 1 if fut['flip_now'].mean() >= 0.5 else 0
            feat['y'] = y
            rows.append(feat)
    return pd.DataFrame(rows)

samples = make_samples(mesh, lags=6, fmin=3, fmax=6)
FEATURES = [c for c in samples.columns if c not in ['lat_bin','lon_bin','y']]
X = samples[FEATURES].values
y = samples['y'].values

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
clf = LogisticRegression(max_iter=400, class_weight='balanced').fit(Xtr, ytr)
probs = clf.predict_proba(Xte)[:,1]
print('AUC:', roc_auc_score(yte, probs).round(3), 'Brier:', brier_score_loss(yte, probs).round(3))

## 4) Flip-Risk Map (live)

In [ ]:
def predict_latest_map(df, clf, features: List[str]):
    rows = []
    for (la,lo), g in df.groupby(['lat_bin','lon_bin']):
        g = g.sort_values(['year','month'])
        g_last = g.tail(6)
        if len(g_last) < 6: 
            continue
        feat = {
            'JellyIdx_last': g_last['JellyIdx'].iloc[-1],
            'JellyIdx_mean': g_last['JellyIdx'].mean(),
            'sst_anom_last': g_last['sst_anom'].iloc[-1],
            'sst_anom_mean': g_last['sst_anom'].mean(),
            'chla_anom_last': g_last['chla_anom'].iloc[-1],
            'chla_anom_mean': g_last['chla_anom'].mean(),
        }
        Xv = np.array([feat[f] for f in features]).reshape(1,-1)
        p = clf.predict_proba(Xv)[:,1][0]
        rows.append({'lat': la, 'lon': lo, 'flip_risk': float(p)})
    return pd.DataFrame(rows)

flipmap = predict_latest_map(mesh, clf, FEATURES)

def plot_map(df, title='Flip-Risk (next 6 months)'):
    pivot = df.pivot_table(index='lat', columns='lon', values='flip_risk', aggfunc='mean').sort_index()
    plt.figure(figsize=(9,6))
    if HAS_CARTOPY:
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([BBOX[0], BBOX[2], BBOX[1], BBOX[3]], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.LAND, edgecolor='black', linewidth=0.4, zorder=2)
        im = ax.imshow(pivot.values, origin='lower',
                       extent=[pivot.columns.min(), pivot.columns.max(), pivot.index.min(), pivot.index.max()],
                       vmin=0, vmax=1, transform=ccrs.PlateCarree())
        plt.colorbar(im, ax=ax, label='Flip-Risk (0..1)')
        ax.set_title(title)
    else:
        im = plt.imshow(pivot.values, origin='lower',
                        extent=[pivot.columns.min(), pivot.columns.max(), pivot.index.min(), pivot.index.max()],
                        vmin=0, vmax=1, aspect='auto')
        plt.colorbar(im, label='Flip-Risk (0..1)')
        plt.title(title); plt.xlabel('Lon'); plt.ylabel('Lat')
    plt.tight_layout(); plt.show()

plot_map(flipmap)

## 5) Hotspot Watch-List

In [ ]:
from scipy.stats import theilslopes

def monthly_risk_series(df_grid, clf, features):
    rows = []
    for p in MONTHS:
        ym = (df_grid['year']<p.year) | ((df_grid['year']==p.year) & (df_grid['month']<=p.month))
        g2 = df_grid[ym]
        pred = predict_latest_map(g2, clf, features)
        pred['time'] = p.to_timestamp(how='end')
        rows.append(pred)
    return pd.concat(rows, ignore_index=True)

risk_series = monthly_risk_series(mesh, clf, FEATURES)

def make_demo_zones(bbox=BBOX):
    lon_splits = np.linspace(bbox[0], bbox[2], 4)
    lat_splits = np.linspace(bbox[1], bbox[3], 3)
    zones = []
    zid = 1
    for i in range(len(lat_splits)-1):
        for j in range(len(lon_splits)-1):
            zones.append({
                'zone_id': f'Z{zid}',
                'lat_min': float(lat_splits[i]), 'lat_max': float(lat_splits[i+1]),
                'lon_min': float(lon_splits[j]), 'lon_max': float(lon_splits[j+1])
            })
            zid += 1
    return pd.DataFrame(zones)

zones = make_demo_zones()

def zone_watchlist(risk_series, zones):
    rows = []
    for _, z in zones.iterrows():
        sel = risk_series[(risk_series['lat']>=z.lat_min)&(risk_series['lat']<z.lat_max)&
                          (risk_series['lon']>=z.lon_min)&(risk_series['lon']<z.lon_max)]
        if sel.empty: 
            continue
        g = sel.groupby('time')['flip_risk'].mean().reset_index()
        if len(g) >= 3:
            x = (g['time'] - g['time'].min()).dt.days.values/30.0
            slope, intercept, _, _ = theilslopes(g['flip_risk'].values, x)
        else:
            slope, intercept = np.nan, np.nan
        rows.append({'zone_id': z.zone_id, 'mean_risk': g['flip_risk'].mean(), 'trend': slope})
    out = pd.DataFrame(rows)
    out['rank'] = out.sort_values(['trend','mean_risk'], ascending=[False,False]).reset_index(drop=True).index+1
    return out.sort_values('rank')

watchlist = zone_watchlist(risk_series, zones)
watchlist

## 6) (Optional) ICES fish index

Add ICES assessment tables by providing `AssessmentKey` values; map to your grid or zones.

In [ ]:
import pandas as pd
import requests

ICES_SVC = 'https://stock-assessment-graphs.ices.dk/services/odata'

def ices_get_stock_download(assessment_key: int) -> pd.DataFrame:
    url = f'{ICES_SVC}/GetStockDownloadData(AssessmentKey={assessment_key})'
    return pd.read_json(url)

ASSESSMENT_KEYS = []  # e.g., [1802, 1786]
ices_tables = []
for k in ASSESSMENT_KEYS:
    try:
        dfk = ices_get_stock_download(k)
        dfk['AssessmentKey'] = k
        ices_tables.append(dfk)
    except Exception as e:
        print('ICES error for', k, ':', e)

if ices_tables:
    ices_df = pd.concat(ices_tables, ignore_index=True)
    display(ices_df.head())
else:
    print('Add keys to ASSESSMENT_KEYS to fetch ICES tables.')

## 7) Save artifacts for slides

In [ ]:
from pathlib import Path
Path('outputs_parviflo_live').mkdir(parents=True, exist_ok=True)
flipmap.to_csv('outputs_parviflo_live/fliprisk_grid.csv', index=False)
watchlist.to_csv('outputs_parviflo_live/hotspot_watchlist.csv', index=False)

# Save map image
plot_map(flipmap, title='Flip-Risk (Live, next 6 months)')
plt.savefig('outputs_parviflo_live/fliprisk_map.png', dpi=160, bbox_inches='tight')
print('Saved files in outputs_parviflo_live/')